In [5]:
# Importing all the necessary packages

import pandas as pd
import numpy as np
import os

In [6]:
directory = os.path.dirname(os.getcwd()) 
path=directory+'/Recommendation_Systems/Data'
#path

In [7]:
item_cols = ['item_id', 'movie_title','release_date','video','release_date','IMDb_URL','unknown', 'Action','Adventure','Animation',"Children's",'Comedy','Crime','Documentary','Drama','Fantasy','Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western']
items = pd.read_csv(path+'/u.item', sep='|',names=item_cols,encoding='latin-1')

items.head(2)

,item_id,movie_title,release_date,video,release_date.1,IMDb_URL,unknown,Action,Adventure,Animation,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,NaN
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,1,0,0,NaN


In [8]:
user_cols = ['user_id', 'item_id', 'rating', 'timestamp']
user = pd.read_csv(path+'/u.data', sep='\t', names=user_cols,encoding='latin-1')

user.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [9]:
dataset = pd.merge(items[['item_id','movie_title']],user[['user_id','rating','item_id']], on='item_id' )
dataset.head()

,item_id,movie_title,user_id,rating
0,1,Toy Story (1995),308,4
1,1,Toy Story (1995),287,5
2,1,Toy Story (1995),148,4
3,1,Toy Story (1995),280,4
4,1,Toy Story (1995),66,3


In [10]:
#dataset[['item_id']].count().drop_duplicates()

In [11]:
#print(dataset.item_id.unique().shape[0])
#print(dataset.user_id.unique().shape[0])

In [12]:
#dataset.movie_title.unique().shape[0]

In [13]:
f = lambda dataset: dataset.sort('item_id', ascending=False)
a=dataset[['item_id','movie_title']].groupby('movie_title').apply(f).reset_index(0,drop=True).drop_duplicates().sort('item_id', ascending=True)

/Users/sonalichaudhari/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':
/Users/sonalichaudhari/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


In [14]:
a.head()

,item_id,movie_title
0,1,Toy Story (1995)
452,2,GoldenEye (1995)
583,3,Four Rooms (1995)
673,4,Get Shorty (1995)
882,5,Copycat (1995)


In [23]:
X = dataset.pivot(index ='user_id',columns='item_id',values='rating').fillna(0)
A = X.head()

In [16]:
# Pearson correlation method
def pearson_corr(x,y):
    x_cord=x-x.mean()
    #print(x,x_cord)
    y_cord=y-y.mean()
    #print(y,y_cord)
    
    num = np.sum(x_cord * y_cord)
    den = np.sqrt(np.sum(x_cord**2) * np.sum(y_cord**2))
    return num/den

In [24]:
def recommendations(name,count):
    id = int(a[a['movie_title']==name]['item_id'])
    r=[]
    w={}
    for e in A.columns:
        if e==id:
        c=pearson_corr(A[id],A[e])
        r.append((e,c))
    r.sort(key=lambda tup: tup[1],reverse=True)
    for t in range(count):
        q = r[t][0]
        title = (a[a['item_id']==q]['movie_title']).to_string(index = False)
        w[title] = r[t][1]
    return(w)

In [22]:
recommendations("B. Monkey (1998)",5)

{'Hurricane Streets (1998)': 0.60252813458919496,
 'Legal Deceit (1997)': 0.4878505366563442,
 "Mat' i syn (1997)": 1.000000000000002,
 'Nil By Mouth (1997)': 0.47054193352980511,
 'Sliding Doors (1998)': 1.000000000000002}

In [20]:
b = (a[a['item_id']==1679]['movie_title']).to_string(index = False)

In [21]:
b

'B. Monkey (1998)'